# Final Project

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this project are the following: 


In [1]:
#intial set of imports
import nltk
from nltk.corpus import wordnet, stopwords
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.wsd import lesk
import pandas as pd
import string
import regex

#variable initialization and instantiation
tests = []
tests_lem = []
gold_std_train = []
gold_std_test = []
lem1 = []
lem2 = []


## 1. Read all pairs of sentences of the train and test set
We proceed to open and read the input file

In [2]:
def text_to_sentences(filename):
    sentence_pair_array = []
    for line in open(filename, encoding="UTF8").readlines():
        sentence_pair_array.append([s.strip() for s in line.split("\t")])
    return sentence_pair_array

# TRIAL TESTING
trial_input = text_to_sentences('./00_data/trial/STS.input_fixed.txt')
trial_classes = open('./00_data/trial/STS_fixed.gs.txt', encoding="utf-8-sig").readlines()

# TRAINING PHASE
train_input = text_to_sentences('./00_data/train/STS.input.MSRpar_vid_SMT.txt')
train_classes = open('./00_data/train/STS.gs.MSRpar_vid_SMT.txt', encoding="utf-8-sig").readlines()

# TESTING PHASE
test_input = text_to_sentences('./00_data/test-gold/STS.input.ALL.txt')
test_classes = open('./00_data/test-gold/STS.gs.ALL.txt', encoding="UTF8").readlines()

We implement the lematizer. 

## 2. Noisy entities removal functions
### Stopwords, URL's, Punctuations

In [3]:
# Removing the punctuation and lowering the case of a string
def preprocessing(line):
    line = line.lower()
    # Clean the text
    line = regex.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", line)
    line = regex.sub(r"what's", "what is ", line)
    line = regex.sub(r"\'s", " ", line)
    line = regex.sub(r"\'ve", " have ", line)
    line = regex.sub(r"can't", "cannot ", line)
    line = regex.sub(r"n't", " not ", line)
    line = regex.sub(r"i'm", "i am ", line)
    line = regex.sub(r"\'re", " are ", line)
    line = regex.sub(r"\'d", " would ", line)
    line = regex.sub(r"\'ll", " will ", line)
    line = regex.sub(r",", " ", line)
    line = regex.sub(r"\.", " ", line)
    line = regex.sub(r"!", " ! ", line)
    line = regex.sub(r"\/", " ", line)
    line = regex.sub(r"\^", " ^ ", line)
    line = regex.sub(r"\+", " + ", line)
    line = regex.sub(r"\-", " - ", line)
    line = regex.sub(r"\=", " = ", line)
    line = regex.sub(r"'", " ", line)
    line = regex.sub(r"(\d+)(k)", r"\g<1>000", line)
    line = regex.sub(r":", " : ", line)
    line = regex.sub(r" e g ", " eg ", line)
    line = regex.sub(r" b g ", " bg ", line)
    line = regex.sub(r" u s ", " american ", line)
    line = regex.sub(r"\0s", "0", line)
    line = regex.sub(r" 9 11 ", "911", line)
    line = regex.sub(r"e - mail", "email", line)
    line = regex.sub(r"j k", "jk", line)
    line = regex.sub(r"\s{2,}", " ", line)

    #only accept alphanum
    # [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    #remove punctuation
    return line.translate(str.maketrans('', '', string.punctuation))

## 3. Word normalization
### Tokenization, Lemmatization

In [4]:
# extract the words from the sentence minus stopwords
def words_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word not in nltk.corpus.stopwords.words('english')]

# convert words to tokens
def tokens_from_words(words):
    return pos_tag(words)

# Function to get wordnet pos code
def wordnet_pos_code(tag):
    if tag.startswith('NN'):
        return wordnet.NOUN
    elif tag.startswith('VB'):
        return wordnet.VERB
    elif tag.startswith('JJ'):
        return wordnet.ADJ
    elif tag.startswith('RB'):
        return wordnet.ADV
    else:
        return None

# Tokens to lemmas using wordnet lemmatizer    
def tokens_to_lemmas(tokens):
    return list(map(token_to_lemmas, tokens))

def token_to_lemmas(token):    
    pos = wordnet_pos_code(token[1])
    if pos:
        return WordNetLemmatizer().lemmatize(token[0], pos=pos)
    return token[0]

## 4. Word Standarization
### Regular Expression 


## 5. Distance Metrics

jaccard distance


## 6. Potential Networks
- MLPRegressor --> Using regressors
- Perceptron
- LinealRegression -->

In [5]:
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron

## 7. Evaluation

In [6]:
# evaluation imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
%matplotlib inline

## Main Program

In [9]:
def lemma_jaccard(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    #print(sent_0 + '\n' + sent_1 + '\n')
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    
    #jaccard_similarity
    #print (float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))), '\n')
    return float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1)))

In [10]:
print('Training')
training_data_X = [lemma_jaccard(data[0], data[1]) for data in train_input]
training_scores_y = [float(line.strip()) for line in train_classes]
print('Finished Training!\n')

print('Testing')
testing_data_X = [lemma_jaccard(data[0], data[1])for data in test_input]
testing_scores_y = [float(line.strip()) for line in test_classes]
print('Finished Testing!\n')

print('Results')
#maybe label encoding https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous
#print(training_data_X)
#print('########################################################################\n')
#print(np.array(training_data_X).reshape(-1,1))
#print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n')
#print(training_scores_Y)


Training
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [11]:
# Creating a Pandas DataFrame
df_X = pd.DataFrame(training_data_X)
df_X.head()

df_y = pd.Series(training_scores_y)
df_y.head() 

df_X_Test = pd.DataFrame(testing_data_X)

In [12]:
from sklearn.neural_network import MLPRegressor
r = MLPRegressor()
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
a = pearsonr(training_scores_y, train_prediction)[0]
b = pearsonr(testing_scores_y, test_prediction)[0]
print('Training Accuracy: ',round(a,3))
print('Testing Accuracy: ', round(b,3))
print('Drop Train-Test: ', round(a-b,3))

Results are ready!

Training Accuracy:  0.69
Testing Accuracy:  0.627
Drop Train-Test:  0.063


/home/jan/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
